In [ ]:
def plot_bathymetry_with_stations(
    topo,
    station_coords,
    selected_station_names=None,
    lon_bounds=(-128.5, -127.5),
    lat_bounds=(51, 52.5),
    deepest=500
):
    """
    Plot bathymetry and stations from data.
    """
    import matplotlib.pyplot as plt
    import cartopy.crs as ccrs
    import cmocean as cm
    import numpy as np

    # Select region
    topo_sel = topo.sel(
        lon=slice(lon_bounds[0], lon_bounds[1]),
        lat=slice(lat_bounds[0], lat_bounds[1])
    )
    depth = -topo_sel['Band1']

    # Plot
    fig, ax = plt.subplots(figsize=(12, 9), subplot_kw={'projection': ccrs.PlateCarree()})
    ax.set_extent([*lon_bounds, *lat_bounds])

    levels = np.linspace(0, deepest, 22)
    cf = ax.contourf(
        topo_sel['lon'], topo_sel['lat'], depth,
        levels=levels, cmap=cm.cm.deep, extend='both'
    )
    contours = ax.contour(
        topo_sel['lon'], topo_sel['lat'], depth,
        levels=levels, colors='k', linewidths=0.3
    )
    ax.clabel(contours, fmt='%d', fontsize=4)
    plt.colorbar(cf, ax=ax, label='Depth (m)')

    # Plot stations
    for lat, lon, name in station_coords:
        # Convert bytes to str and strip spaces
        if isinstance(name, bytes):
            name = name.decode().strip()
        else:
            name = str(name).strip()
        
        if (selected_station_names is None or name in selected_station_names):
            if lat_bounds[0] <= lat <= lat_bounds[1] and lon_bounds[0] <= lon <= lon_bounds[1]:
                ax.plot(
                    lon, lat,
                    marker='o', color='red', markersize=6,
                    transform=ccrs.PlateCarree()
                )
                ax.text(
                    lon + 0.01, lat,
                    name, fontsize=9,
                    transform=ccrs.PlateCarree()
                )

    ax.set_title("Bathymetry with Stations")
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")

    plt.tight_layout()

plot_bathymetry_with_stations(
    topo=topo,
    station_coords=station_coords,
    selected_station_names=["FZH01", "FZH04", "FZH07", "FZH08", "FZH013", "FZH014"],
    lon_bounds=(-128.5, -127.5),
    lat_bounds=(51, 52.5),
    deepest=500)